In [8]:
import pandas as pd
from feed_forward_nn import Model, PassRunDataset, test
import torch
from torch import nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
model_data = pd.read_csv("test_results.csv")
model_data

,Model,Accuracy,HiddenDims,HiddenLayers,PValue,LearningRate,Epochs
0,7afc6,70.077558,64,1,0.001,0.0100,16
1,81e39,71.078309,64,1,0.001,0.0010,41
2,a49bc,68.101076,64,1,0.001,0.0001,56
3,9aeaa,69.902427,64,1,0.050,0.0100,26
4,c00e6,69.939955,64,1,0.050,0.0010,56
...,...,...,...,...,...,...,...
76,85b14,70.502877,256,3,0.050,0.0010,26
77,64fac,69.176883,256,3,0.050,0.0001,26
78,1362b,69.226920,256,3,1.000,0.0100,21
79,3358f,68.326245,256,3,1.000,0.0010,31


In [4]:
model_data.sort_values("Accuracy", ascending=False)

,Model,Accuracy,HiddenDims,HiddenLayers,PValue,LearningRate,Epochs
58,8f007,71.278459,256,1,0.050,0.0010,31
1,81e39,71.078309,64,1,0.001,0.0010,41
66,01877,70.815612,256,2,0.050,0.0100,36
19,c5ad7,70.790593,64,3,0.001,0.0010,31
64,e0ede,70.765574,256,2,0.001,0.0010,16
...,...,...,...,...,...,...,...
14,257ee,68.288717,64,2,0.050,0.0001,56
11,4b15d,68.226170,64,2,0.001,0.0001,46
2,a49bc,68.101076,64,1,0.001,0.0001,56
35,ddfbd,67.875907,128,1,1.000,0.0001,36


In [9]:

features = pd.read_csv("./data/scaled_features.csv")
cols = ['down', 'no_huddle', 'goal_to_go', 'defteam_score', 'half_seconds_remaining', 'quarter_seconds_remaining', 'posteam_timeouts_remaining', 'score_differential', 'ydstogo', 'posteam_score', 'game_seconds_remaining', 'total_away_score', 'away_timeouts_remaining', 'yardline_100', 'home_timeouts_remaining', 'drive']
features = features[cols]
labels = pd.read_csv("./data/labels.csv")

test_data = PassRunDataset(features, labels)
test_dataloader = torch.utils.data.DataLoader(
    test_data, batch_size=256, shuffle=True
)


best_model = Model(input_size=test_data.n_features(), output_size=1, hidden_dim=256, n_layers=1)

load_path = 'checkpoints/8f007'
best_model.load_state_dict(torch.load(load_path))

best_model.to(best_model.device)

lf = nn.BCEWithLogitsLoss()

_, acc, y, pred = test(dataloader=test_dataloader, model=best_model, lf=lf)

acc

71.05863956215794

In [10]:
conf = confusion_matrix(y, pred)
pr, r, f, s = precision_recall_fscore_support(y, pred, pos_label=1, average='binary')

print("Precision: {:.3f} Recall: {:.3f} F1-Score: {:.3f}".format(pr, r, f))
print(conf)

Precision: 0.764 Recall: 0.742 F1-Score: 0.753
[[ 8634  4360]
 [ 4894 14087]]
